In [1]:
from lightgbm import LGBMRegressor
from sklearn.compose import make_column_transformer
from sklearn.linear_model import TheilSenRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

import pandas as pd
import numpy as np

In [2]:
# I want my cells to look pretty automatically.
%load_ext lab_black

# Data source

We have downloaded the January 2016 from the [New York City Taxi & Limousine Commission Trip Record Data](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page) and converted it into an Apache Parquet file using the following code:

```
df = pd.read_csv(
    "../data/yellow_tripdata_2016-01.csv",
    dtype={"store_and_fwd_flag": "bool"},
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    index_col=False,
    infer_datetime_format=True,
    true_values=["Y"],
    false_values=["N"],
)
df.to_parquet("../data/yellow_tripdata_2016-01.parquet")
```

This is a really [nice dataset to demonstrate a lot of things as outlined in a previous blog post](https://uwekorn.com/2019/08/22/why-the-nyc-trd-is-a-nice-training-dataset.html).

In [3]:
df = pd.read_parquet("../data/yellow_tripdata_2016-01.parquet")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10906858 entries, 0 to 10906857
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        int64         
 4   trip_distance          float64       
 5   pickup_longitude       float64       
 6   pickup_latitude        float64       
 7   RatecodeID             int64         
 8   store_and_fwd_flag     bool          
 9   dropoff_longitude      float64       
 10  dropoff_latitude       float64       
 11  payment_type           int64         
 12  fare_amount            float64       
 13  extra                  float64       
 14  mta_tax                float64       
 15  tip_amount             float64       
 16  tolls_amount           float64       
 17  improvement_surcharge  float64       
 18  total_amount        

In [5]:
# Split data into 80% training and 20% test data
df_train, df_test = train_test_split(df, train_size=0.8)

In [6]:
# Remove basic outliers from the training data
cap_fare = df_train["fare_amount"].mean() + 3 * df_train["fare_amount"].std()
cap_distance = df_train["trip_distance"].mean() + 3 * df_train["trip_distance"].std()
df_train = df_train.query(
    f"trip_distance > 0 and trip_distance < {cap_distance} and fare_amount > 0 and fare_amount < {cap_fare}"
)

In [7]:
# Create a 5% sample dataset that runs much faster

df_sampled = df_train.sample(frac=0.05)
y_sampled = df_sampled.pop("fare_amount")
y_train = df_train.pop("fare_amount")
y_test = df_test.pop("fare_amount")

# Simple regression baseline

As we are training a regression model here and we want to be sure it performs reasonable well, we should first calculate a baseline with a very simple model.
We then use this to benchmark all other models against it.

We select the mean absolute error as our cost function as absolute dollars is a more tangible and realistic measure than square dollars. The most simple baseline for this cost function is to take the median of all values in the training data.

In [8]:
average_fare = y_train.median()
(y_test - average_fare).abs().mean(), (y_train - average_fare).abs().mean()

(6.11231579024577, 6.004128783019656)

# Simple Feature Engineering

The dataset is quite rich in features but at the start of a journey, they are realistically not all known when you start a journey. Thus we will limit ourselves to a very basic set.

In [9]:
def haversine_distance(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = (np.radians(x) for x in (lat1, lng1, lat2, lng2))
    d = (
        np.sin(lat2 / 2 - lat1 / 2) ** 2
        + np.cos(lat1) * np.cos(lat2) * np.sin(lng2 / 2 - lng1 / 2) ** 2
    )
    return 2 * 6371 * np.arcsin(np.sqrt(d))  # 6,371 km is the earth radius


column_transforms = [
    (FunctionTransformer(), ["passenger_count"]),
    (
        FunctionTransformer(
            func=lambda df: pd.DataFrame(
                {
                    "pickup_dayofweek": df["tpep_pickup_datetime"].dt.dayofweek,
                    "pickup_hour": df["tpep_pickup_datetime"].dt.hour,
                    "pickup_minute": df["tpep_pickup_datetime"].dt.minute,
                }
            )
        ),
        ["tpep_pickup_datetime"],
    ),
    (
        FunctionTransformer(
            func=lambda df: pd.DataFrame(
                {
                    "haversine_distance": haversine_distance(
                        df["pickup_latitude"],
                        df["pickup_longitude"],
                        df["dropoff_latitude"],
                        df["dropoff_longitude"],
                    )
                }
            )
        ),
        [
            "pickup_latitude",
            "pickup_longitude",
            "dropoff_latitude",
            "dropoff_longitude",
        ],
    ),
]

Instead of using ordinary least squares (OLS) which optimizes the L2 norm, we are taking as the first model approach the [Theil-Sen Regressor](https://scikit-learn.org/stable/auto_examples/linear_model/plot_theilsen.html). Due to its computational complexity, we only train it on 5% of the data.


In [10]:
%%time
tsr = make_pipeline(make_column_transformer(*column_transforms), TheilSenRegressor())
tsr.fit(df_sampled, y_sampled)

CPU times: user 1min 50s, sys: 16 s, total: 2min 6s
Wall time: 2min 13s


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('functiontransformer-1',
                                                  FunctionTransformer(),
                                                  ['passenger_count']),
                                                 ('functiontransformer-2',
                                                  FunctionTransformer(func=<function <lambda> at 0x107cd68b0>),
                                                  ['tpep_pickup_datetime']),
                                                 ('functiontransformer-3',
                                                  FunctionTransformer(func=<function <lambda> at 0x15db98b80>),
                                                  ['pickup_latitude',
                                                   'pickup_longitude',
                                                   'dropoff_latitude',
                                                   'dropoff_longitude'])])),
  

In [11]:
(tsr.predict(df_train) - y_train).abs().mean(), (
    tsr.predict(df_test) - y_test
).abs().mean()

(53.22234508455122, 84.59079127235368)

As the Theil-Sen regressor sadly performs much worse than our above baseline, we use instead LightGBM with L1 as the objective function. While this introduces a new dependency, it is a well-proven technique and as the metric shows, also an effective one in this approach.

In [12]:
lgbm_pipeline = make_pipeline(
    make_column_transformer(*column_transforms),
    LGBMRegressor(objective="regression_l1"),
)

In [13]:
lgbm_pipeline.fit(df_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('functiontransformer-1',
                                                  FunctionTransformer(),
                                                  ['passenger_count']),
                                                 ('functiontransformer-2',
                                                  FunctionTransformer(func=<function <lambda> at 0x107cd68b0>),
                                                  ['tpep_pickup_datetime']),
                                                 ('functiontransformer-3',
                                                  FunctionTransformer(func=<function <lambda> at 0x15db98b80>),
                                                  ['pickup_latitude',
                                                   'pickup_longitude',
                                                   'dropoff_latitude',
                                                   'dropoff_longitude'])])),
  

In [14]:
(lgbm_pipeline.predict(df_train) - y_train).abs().mean()

1.9753373838193955

In [15]:
(lgbm_pipeline.predict(df_test) - y_test).abs().mean()

2.1114189266420054